In [5]:
import requests
import json
import pandas as pd
from datetime import datetime

# Point to correct subgraph URL
url = 'https://api.thegraph.com/subgraphs/name/benesjan/uniswap-v3-subgraph'

# initialize full_data
df1_fulldata = pd.DataFrame()
df2_fulldata = pd.DataFrame()
df_data = pd.DataFrame()

for i in range(0,6) :
    # Set query (which uses text input to specify ETH wallet addresses)
    query = '''query {
      swaps(orderBy: timestamp orderDirection: desc, first:1000, skip:%s){
        id
        timestamp
        pool{
          token0{
            symbol
          }
          token1{
            symbol
          }
          txCount
        }
        amount0
        amount1
        amountUSD
        token0{
            id
            totalSupply
            volume
            volumeUSD
            totalValueLocked
            totalValueLockedUSD
        }
        token1{
            id
            totalSupply
            volume
            volumeUSD
            totalValueLocked
            totalValueLockedUSD
        }
      }
    }
    ''' % (i*1000)


    # Make the request
    r = requests.post(url, json={'query': query})
    #print(r.status_code)
    #print(r.text)
    
    # JSON adjustment
    json_data = json.loads(r.text)
    
    # extract JSON to convert to a dataframe
    df_data = json_data['data']['swaps']
    df1_fulldata = pd.DataFrame(df_data)
        
    df2_fulldata = pd.concat([df2_fulldata,df1_fulldata])
    
#print(df2_fulldata)  

#EDA
#df1.to_excel(r'C:\Users\Nehal.Jain\Documents\Nehal_Personal\Project\Blockchain\Dataset.xlsx', sheet_name='Data', index = False)

#Removing NULLS - Data Cleaning
df = df2_fulldata.dropna()
#print(df)

#print(df.isnull().values.any())
#print(df.isnull().sum().sum())

#Converting from Unix timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')

#Splitting 'pool' column into 'From_Currency' and 'To_Currency'

df[['From_Currency','To_Currency','txCount']] = df.pool.apply(
   lambda x: pd.Series(str(x).split(",")))

#Splitting 'Token' columns
df[['From_Token_id','From_Token_totalSupply','From_Token_totalValueLocked','From_Token_totalValueLockedUSD','From_Token_volume','From_Token_volumeUSD']] = df.token0.apply(
   lambda z: pd.Series(str(z).split(",")))
df[['To_Token_id','To_Token_totalSupply','To_Token_totalValueLocked','To_Token_totalValueLockedUSD','To_Token_volume','To_Token_volumeUSD']] = df.token1.apply(
   lambda n: pd.Series(str(n).split(",")))

#Removing extra data from columns From_Currency and To_Currency : Cleaning
df['From_Currency'] = df['From_Currency'].map(lambda x: str(x)[23:-2])
df['To_Currency'] = df['To_Currency'].map(lambda y: str(y)[23:-2])
df['txCount'] = df['txCount'].map(lambda y: str(y)[13:-2])
df['From_Token_id'] = df['From_Token_id'].map(lambda y: str(y)[8:-1])
df['To_Token_id'] = df['To_Token_id'].map(lambda y: str(y)[8:-1])
df['From_Token_totalSupply'] = df['From_Token_totalSupply'].map(lambda y: str(y)[17:-1])
df['To_Token_totalSupply'] = df['To_Token_totalSupply'].map(lambda y: str(y)[17:-1])
df['From_Token_totalValueLocked'] = df['From_Token_totalValueLocked'].map(lambda y: str(y)[22:-1])
df['From_Token_totalValueLockedUSD'] = df['From_Token_totalValueLockedUSD'].map(lambda y: str(y)[25:-1])
df['To_Token_totalValueLocked'] = df['To_Token_totalValueLocked'].map(lambda y: str(y)[22:-1])
df['To_Token_totalValueLockedUSD'] = df['To_Token_totalValueLockedUSD'].map(lambda y: str(y)[25:-1])
df['From_Token_volume'] = df['From_Token_volume'].map(lambda y: str(y)[12:-1])
df['From_Token_volumeUSD'] = df['From_Token_volumeUSD'].map(lambda y: str(y)[15:-2])
df['To_Token_volume'] = df['To_Token_volume'].map(lambda y: str(y)[12:-1])
df['To_Token_volumeUSD'] = df['To_Token_volumeUSD'].map(lambda y: str(y)[15:-2])

#print(df)

#Dropping the pool, token columns
df = df.drop(['pool'], axis=1)
df = df.drop(['token0'], axis=1)
df = df.drop(['token1'], axis=1)

#Cleaning and re-ordering of data
df = df.rename(columns={'amount0': 'From_Currency_Amount', 'amount1': 'To_Currency_Amount', 'amountUSD': 'Amount_USD', 'id': 'TransactionID','timestamp': 'Date_Time'})
df = df [['TransactionID','From_Token_id','From_Currency_Amount','From_Currency','From_Token_totalSupply','From_Token_totalValueLocked','From_Token_totalValueLockedUSD','From_Token_volume','From_Token_volumeUSD','To_Token_id','To_Currency_Amount','To_Currency','To_Token_totalSupply','To_Token_totalValueLocked','To_Token_totalValueLockedUSD','To_Token_volume','To_Token_volumeUSD','Amount_USD','txCount','Date_Time']]
df.head()
#print(df)


,TransactionID,From_Token_id,From_Currency_Amount,From_Currency,From_Token_totalSupply,From_Token_totalValueLocked,From_Token_totalValueLockedUSD,From_Token_volume,From_Token_volumeUSD,To_Token_id,To_Currency_Amount,To_Currency,To_Token_totalSupply,To_Token_totalValueLocked,To_Token_totalValueLockedUSD,To_Token_volume,To_Token_volumeUSD,Amount_USD,txCount,Date_Time
0,0x99477adea8ad6316ca22fefd05fa3cc9ef651c863282...,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2498.836039,USDC,19312,452667772.911958,452667772.9119580000000000000000001,50069000791.52052,50070095464.68774091650185991160817,0xdac17f958d2ee523a2206206994597c13d831ec7,-2497.332924,USDT,18240,164015194.948309,164009312.8810858063805959332755454,18963086229.05562,18977781922.30131973582535094711578,2498.039700648621284094617771547925,91196,2021-08-03 15:53:00
1,0x99477adea8ad6316ca22fefd05fa3cc9ef651c863282...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,-1,WETH,19848,278609.726822882039101023,695502301.381092008387879760314504,30307362.797758437179914022,72531552754.98649273415144931560682,0xdac17f958d2ee523a2206206994597c13d831ec7,2497.332924,USDT,18240,164015194.948309,164009312.8810858063805959332755454,18963086229.05562,18977781922.30131973582535094711578,2496.787546196177922613813549012962,260396,2021-08-03 15:53:00
2,0x3ab3ee348332b7685347405147d2fb48e72cbde9fde0...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0.249416394260634144,WETH,19848,278609.726822882039101023,695502301.381092008387879760314504,30307362.797758437179914022,72531552754.98649273415144931560682,0xcc8fa225d80b9c7d42f96e9570156c65d6caaa25,-3000,SLP,19312,11179485,2313417.787046968992848968194971909,1546556003,354503905.5690793228317777319860726,622.626058998733714193817508734648,95821,2021-08-03 15:53:00
3,0x2a3a50024131acdd1382b8a12ba0658a5b8899ce35ac...,0x6de037ef9ad2725eb40118bb1702ebb27e4aeb24,-2760.273655958616918552,RNDR,18240,225549.92709121570907978,0,7086357.844595524237178251,4617863.251262753647885577419236545,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0.78,WETH,19848,278609.726822882039101023,695502301.381092008387879760314504,30307362.797758437179914022,72531552754.98649273415144931560682,1947.138749474188356089945412845458,2551,2021-08-03 15:53:00
4,0xfa1d3f65dc290e51d76f592a0be21dbfbb601f481071...,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,1.829,WBTC,18240,4113.22678566,156626182.3952428554861103688544275,229972.62902799,8571800308.819774602305347747761836,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,-27.882757032984357747,WETH,19848,278609.726822882039101023,695502301.381092008387879760314504,30307362.797758437179914022,72531552754.98649273415144931560682,69625.2441509004976547987277403229,61114,2021-08-03 15:52:54


In [6]:
print(df)

                                         TransactionID  \
0    0x99477adea8ad6316ca22fefd05fa3cc9ef651c863282...   
1    0x99477adea8ad6316ca22fefd05fa3cc9ef651c863282...   
2    0x3ab3ee348332b7685347405147d2fb48e72cbde9fde0...   
3    0x2a3a50024131acdd1382b8a12ba0658a5b8899ce35ac...   
4    0xfa1d3f65dc290e51d76f592a0be21dbfbb601f481071...   
5    0x537eb936fcd61b9022a5c06e0df34982d8bd6e217ca0...   
6    0xe04c44fdb4a5bfd8349ce1c773c8097094b32dd30f79...   
7    0xe04a54730ca7aac7277c00c0e5960ee69ed7ce0f3c62...   
8    0xb35cb5b48e02157d36e9da46b75746d2fd6bc271b00a...   
9    0x9c44f7c40081371f6ead49959009afc13e63168ebdb5...   
10   0x53c438f65ce09d12604d8b366cd880750f552b8b837d...   
11   0xeca9c58312f3d636ee4080da31bfe06541cc165df55f...   
12   0xd4ef6ea98428e999200d6d716217a5c958fca4ad584f...   
13   0x42a0b27b04faff4bcc0ba0c61586439cf893c913743c...   
14   0x42a0b27b04faff4bcc0ba0c61586439cf893c913743c...   
15   0x420dceb3f8b423cb59cb26ae918c1f587d66abca256a...   
16   0x34514c2